In [3]:
import pandas as pd
import numpy as np
import re
from multiprocessing import Pool
import scattertext as st
import spacy
import pickle
%matplotlib inline

In [9]:
# df_parsed = pd.read_pickle('df_parsed.pkl')
# df_parsed.set_index('Date_',inplace=True)

# top20froms = (lambda x: set(x[:20].index))(df_parsed['From'].value_counts())

# df_parsed['FromCat'] = df_parsed.From.apply(lambda x: x if x in top20froms else 'other')

nlp = spacy.load('en')

# corpus = st.CorpusFromPandas(df_parsed.sample(10000), 
#                               category_col='FromCat', 
#                               text_col='Body',
#                               nlp=nlp).build()

corpus = st.CorpusFromPandas(df_parsed.sample(10000), 
                              category_col='FromCat', 
                              text_col='Body',
                              nlp=nlp).build()

## Significant Words Using ScatterText

In [5]:
corpus = pickle.load(open('corpus.p','rb'))

# df_parsed_sample.to_pickle('df_parsed_sample.pkl')

print(list(corpus.get_scaled_f_scores_vs_background().index[:10]))

scaled_f_scores_vs_background = corpus.get_scaled_f_scores_vs_background()

term_freq_df = corpus.get_term_freq_df()

for c in term_freq_df:
    f = c[:c.find(' freq')]
    term_freq_df[f+' fscore'] = corpus.get_scaled_f_scores(f)

columns = pd.Series([x[:x.find(' ')] for x in term_freq_df.columns]).drop_duplicates()

term_freq_df.columns = pd.MultiIndex.from_product([['freq','fscore'],columns])

In [16]:
term_freq_df.loc[:,'fscore'].sort_values('other',ascending=False).iloc[:200].style.background_gradient(subset=columns[1:])

In [45]:
# term_freq_df.loc[:,'fscore'].sort_values('other',ascending=False).iloc[:200].to_excel('fscores.xlsx')

# Entity Extraction

### Polyglot

In [18]:
from polyglot.text import Text
import dask.dataframe as dd
from dask.multiprocessing import get

In [19]:
df_parsed_sample = pd.read_pickle('df_parsed_sample.pkl')

In [20]:
df_parsed_sample.shape

(10000, 22)

In [61]:
df_parsed_sample_with_entities = pd.read_pickle('df_parsed_sample_with_entities.pkl')

In [6]:
def extract_entity(blob):
    try:
        text = Text(blob)
        ents = text.entities
        entities = [' '.join(e) for e in ents]
        tags = [e.tag for e in ents]
        return list(zip(entities,tags))
    except:
        return []

In [7]:
def extract_entities(df):
    return df.Body.apply(extract_entity)

In [8]:
ddf = dd.from_pandas(df_parsed_sample, npartitions=6)
df_parsed_sample['entities'] = ddf.map_partitions(extract_entities, meta=pd.Series()).compute(get=get)

### Spacy

In [54]:
import spacy

nlp = spacy.load('en_core_web_sm')
nlp.disable_pipes('tagger')
nlp.disable_pipes('parser')

docs = [doc for doc in nlp.pipe(df_parsed_sample.Body, n_threads=12)]

entities = [[(ent.label_, ent.text )for ent in doc.ents] for doc in docs]

df_parsed_sample_with_entities['entities_spacy'] = entities

# df_parsed_sample_with_entities.to_pickle('df_parsed_sample_with_entities.pkl')

In [73]:
df_parsed_sample_with_entities.sample()

index               Date_  \
6425   6425 2001-12-20 18:05:12   

                                         Message-ID  \
6425  <25522372.1075855352010.JavaMail.evans@thyme>   

                                       Date                       From  \
6425  Thu, 20 Dec 2001 10:05:12 -0800 (PST)  weather@inbox.weather.com   

                          To  \
6425  mike.grigsby@enron.com   

                                                Subject Mime-Version  \
6425  Mike, here is your weather forecast for Housto...          1.0   

                      Content-Type Content-Transfer-Encoding  \
6425  text/plain; charset=us-ascii                      7bit   

                            ...                          \
6425                        ...                           

                                               X-Folder   X-Origin  \
6425  \Michael_Grigsby_Jan2002_1\Grigsby, Mike\Delet...  Grigsby-M   

                        X-FileName  \
6425  mgrigsb (Non-Privileged).pst   

                                                   Body Bcc Cc  \
6425  <HTML>\n<HEAD>\n\t<TITLE>Weather Inbox</TITLE>...          

                              file FromCat  \
6425  grigsby-m/deleted_items/235.   other   

                                               entities  \
6425  [(Houston, I-LOC), (TX, I-LOC), (houston, I-LO...   

                                         entities_spacy  
6425  [(GPE, \n), (ORG, Weather Inbox</TITLE), (GPE,...  

[1 rows x 24 columns]

In [9]:
def get_topn_by_cat(df, cat, topncats=10):
    topncats = set((lambda x: x[:topncats].index)(df[cat].value_counts()))
    df[cat+'_'] = df[cat].apply(lambda x: x if x in topncats else 'other')
    entities_df = df.groupby(cat+'_').entities.sum().reset_index()

    entities_df_expanded = pd.DataFrame()
    for i, df_t in entities_df.iterrows():
        df_e = pd.DataFrame(df_t.entities, columns=['entity','tag'])
        df_e[cat+'_'] = df_t[cat+'_']
        entities_df_expanded = entities_df_expanded.append(df_e, ignore_index=True)
    def get_top100(df):
        df_t = df.pivot_table(index='entity',columns='tag',aggfunc=len).fillna(0).astype(int).reset_index()
        df_t.columns = ['entity','I-LOC','I-ORG','I-PER']
        df_t['All'] = df_t.select_dtypes(['int']).sum(1)
        df_t.sort_values('All', inplace=True, ascending=False)

        return df_t.iloc[:100]
    ents_by_cat = entities_df_expanded.groupby(cat+'_').apply(get_top100)
    return ents_by_cat.reset_index(level=[0]).reset_index(drop=True)

In [12]:
df_parsed_sample_with_entities['Year'] = df_parsed_sample_with_entities.Date_.dt.year.fillna(0).astype(int)
df_parsed_sample_with_entities['Month'] = df_parsed_sample_with_entities.Date_.dt.month.fillna(0).astype(int)
df_parsed_sample_with_entities['MonthYear'] = pd.to_datetime(df_parsed_sample_with_entities.Date_.dt.strftime('%Y-%m-01'))

In [13]:
ents_by_year = get_topn_by_cat(df_parsed_sample_with_entities, 'Year').set_index('Year_')
ents_by_month = get_topn_by_cat(df_parsed_sample_with_entities, 'Month').set_index('Month_')
ents_by_monthyear = get_topn_by_cat(df_parsed_sample_with_entities, 'MonthYear').set_index('MonthYear_')

In [14]:
ents_by_year.to_excel('ents_by_month.xlsx')
ents_by_monthyear.to_excel('ents_by_monthyear.xlsx')